# Radioactive Decay Interactives

This notebook has embedded in it code for interactive investigation of radioative decay.  

The eventual goal is the development of two ipywidgets:
- Radioactive Decay model showing decay of a population of atoms over time. **[Sam is continuing development of this one.]**
- Isochron dating model **[I (Juan) am tackling this one.]**


In [ ]:
# UNcomment to turn on autoreloading with reach execution
# %load_ext autoreload
# %autoreload 2

from IPython.display import display
import numpy as np
import bqplot as bq
import ipywidgets as widgets
import pandas as pd
from math import ceil, floor


Assuming a non-radiogenic isotope (that is, an isotope that is not the result of radioactive decay) that also will not decay, its amount should be constant.  This means that for different
mineral samples we can measure the ratio of parent isotope versus the non-radiogenic isotope (P/D_i) and daughter isotope (D) versus the non-radiogenic isotope (D/D_i) to build an isochron plot.  For example, using the following isotopes
- Sr-86 (non-radiogenic isotope of daughter element, called D_i)
- Sr-87 (Daughter Isoptope, called D)
- Rb-87 (Parent isotope, called P)
an isochron plot would plot (Sr-87/Sr-86 or D/D_i) versus (Rb-87/Sr-86 or P/D_i).  

What sets the isochron method apart from the just measuring parent and daughter abundances is the use 
of the non-radiogenic isotope of the daughter element.  This avoids the assumption of no initial daughter isotope before the rock solidified (radioactive decay can occur while rock is molten).

Some minerals in the rock incoprorate parent better than daughter which is why the initial amount of parent 
isotope versus daughter isotope can vary.  We expect daughter versus non-radiogenic isotope ratio to be constant
if we pick the non-radiogenic isotope to be the same element as the daughter isotope.

*Note:* The idea for this app came from a Isochron Diagram Java app at ScienceCourseware.org.  However that app had
some issues in that it didn't divide by a non-radiogenic isotope (or at least didn't mention it).  In fact, they
used D_i for the initial amount of daughter isotope instead of the non-radiogenic isotope of the same element
as the daughter isotope.

In [ ]:
##
## Information on Sr-87 and Rb-87
tau_Rb = 4.923e10    # Half-life (in years) of Rb-87 to Sr-87 via beta decay
Earth_age = 4.4e9  # Age of oldest rock on Earth (in years)

# Largest possible fraction of decay (may use this later)
Max_half_lives = 5
Max_decay_fraction = 1 - (1/2)**(Max_half_lives)
Max_half_lives_real = Earth_age/tau_Rb
Max_decay_fraction_real = 1 - (1/2)**(Max_half_lives_real)

# Range of P to D_i fractions and initial amounts of D to D_i to consider
P2Di_min = 0.05
P2Di_max = 0.40
D2Di0_min = 0.05
D2Di0_max = 0.75

# Generate three mineral samples in different thirds of the entire range
range_P2Di  = (P2Di_max-P2Di_min)

# Create sample amounts
n_samples = 4
nums = np.array(list(range(1, n_samples+1)))
initial_samples = pd.DataFrame(index=nums)
initial_D2Di0 = D2Di0_min + (D2Di0_max - D2Di0_min) * np.random.random()
initial_samples['P2Di'] = P2Di_min + (range_P2Di/n_samples) * (nums - np.random.random(n_samples))
initial_samples['D2Di'] = initial_D2Di0*np.ones_like(nums)


In [ ]:
def amt_left(sample_in, taus):
    # Generate a sample DataFrame given an initial DataFrame
    sample = sample_in.copy(deep = True)
    sample['P2Di'] = sample_in['P2Di']*((1/2)**(taus))
    sample['D2Di'] = sample_in['D2Di'] + sample_in['P2Di']*(1 - (1/2)**(taus))
    return sample

def line_points(sample):
    global x_min, x_max, y_min, y_max, initial_D2Di0
    
    # Determine the end points of a line going through the sample points.
    x_range = x_max - x_min
    y_range = y_max - y_min
    
    # Slope (extrapolate from first two points - could be done by a fit to the points)
    slope = (sample['D2Di'][2]-sample['D2Di'][1])/(sample['P2Di'][2]-sample['P2Di'][1])
    y_final = initial_D2Di0 + slope*x_range
    x_points = (x_min, x_max)
    y_points = (initial_D2Di0, y_final)
    return x_points, y_points, slope

def init2current(samples0, samples):
    # Compute the lines connecting initital and final points for plotting
    n_pts = len(samples0)

    xlist = []
    ylist = []
    for pt in range(1, n_pts+1):
        x = np.array([ samples0['P2Di'][pt], samples['P2Di'][pt] ])
        y = np.array([ samples0['D2Di'][pt], samples['D2Di'][pt] ])
        xlist.append(x)
        ylist.append(y)
    
    return(xlist, ylist)
    
def HL_changed(change):
    global isotope, tau_Rb, sample, initial_samples, dots_current, line_current, connectors, slope_label
    
    # How many half-lives have passed?  Use this to get new sample and line info
    this_tau = HL_slider.value
    sample = amt_left(initial_samples, this_tau)
    x_sample, y_sample, slope =  line_points(sample)
    
    # Update plot
    dots_current.x = sample['P2Di']
    dots_current.y = sample['D2Di']
    line_current.x = x_sample
    line_current.y = y_sample
    slope_label.value = 'Slope: {0:.2f}'.format(slope)
    xlist, ylist = init2current(initial_samples, sample)
    connectors.x = xlist
    connectors.y = ylist
    
    if (isotope.value == "Generic"):
        HLlabel.value='{0:.2f} Half-lives'.format(this_tau)
    elif (isotope.value == "Rb/Sr"):
        HLlabel.value='{0:.2f} million years'.format(this_tau*tau_Rb/1e6)
    else:  # Shouldn't happen, but default to the Generic
        HLlabel.value='{0:.2f} Half-lives'.format(this_tau)
        
def isotope_changed(change):
    global ax_x_P2Di, ax_y_D2Di, HL_slider, tau_Rb
    # Switch from generic to Rb/Sr
    if (change.new == "Generic"):
        ax_x_P2Di.label = 'P / D_i'
        ax_y_D2Di.label = 'D / D_i'
        HLlabel.value='{0:.2f} Half-lives'.format(HL_slider.value)
        HL_slider.description = "Half-lives"
    elif (change.new == "Rb/Sr"):
        ax_x_P2Di.label = 'Rb-87 / Sr-86'
        ax_y_D2Di.label = 'Sr-87 / Sr-86'
        HLlabel.value='{0:.2f} million years'.format(HL_slider.value*tau_Rb/1e6)
        HL_slider.description = "Time"
    else:  # Shouldn't happen, but default to the Generic
        ax_x_P2Di.label = 'P / D_i'
        ax_y_D2Di.label = 'D / D_i'
        HLlabel.value='{0:.2f} Half-lives'.format(HL_slider.value)
        HL_slider.description = "Half-lives"

In [ ]:
##
## Set up isochron plot
##

# detemine maximum and minimum of X and Y axes
x_step = 0.05
x_min = 0
x_max = x_step * ceil(initial_samples['P2Di'][n_samples] / x_step)
y_step = 0.04
y_min = y_step * floor(initial_D2Di0 / y_step)
y_max = y_step * ceil((initial_D2Di0 + initial_samples['P2Di'][n_samples] * Max_decay_fraction) / y_step)

# Labels and scales for Axes
x_P2Di = bq.LinearScale(min = x_min, max = x_max)
y_D2Di = bq.LinearScale(min = y_min, max = y_max)
ax_x_P2Di = bq.Axis(label='P / D_i', scale=x_P2Di)
ax_y_D2Di = bq.Axis(label='D / D_i', scale=y_D2Di, orientation='vertical')

# Set up initial conditions
taus = 0    # zero half lives past
sample = amt_left(initial_samples, taus)

##
## Define the lines
##
# Initial amount of daughter line (with dots for initial amounts of parent)
x_init, y_init, slope_init =  line_points(initial_samples)
line_initial = bq.Lines(x=x_init, y=y_init, scales={'x': x_P2Di, 'y': y_D2Di}, 
                   line_style='dashed', colors=['red'], labels=['Initial Sample'])
dots_initial = bq.Scatter(x=initial_samples['P2Di'], y=initial_samples['D2Di'], scales={'x': x_P2Di, 'y': y_D2Di}, 
                   colors=['white'], stroke='red', fill= True, labels=['Initial Isochron'])

# Current quantities on isochron line
x_sample, y_sample, slope =  line_points(sample)
line_current = bq.Lines(x=x_sample, y=y_sample, scales={'x': x_P2Di, 'y': y_D2Di}, 
                   line_style='solid', colors=['red'], labels=['Current Isochron'])
dots_current = bq.Scatter(x=sample['P2Di'], y=sample['D2Di'], scales={'x': x_P2Di, 'y': y_D2Di}, 
                   colors=['red'], stroke='red', fill= True, labels=['Initial Isochron'])

# Connect Initial and Current quantities on isochron line
xlist, ylist = init2current(initial_samples, sample)
connectors = bq.Lines(x=xlist, y=ylist, scales={'x': x_P2Di, 'y': y_D2Di}, 
                   line_style='dotted', colors=['black'])

# Construct plot
isochron = bq.Figure(axes=[ax_x_P2Di, ax_y_D2Di], marks=[connectors, line_initial, dots_initial, line_current, dots_current],
                     title='Isochron Diagram', layout={'width': '700px', 'min_height': '400px'})

# Slider controling number of half-lives
HL_slider = widgets.FloatSlider(value=0, min=0, max=Max_half_lives, step=0.02,
                                description='Half-lives', disabled=False,
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.2f',
                                layout=widgets.Layout(align_content='center', align_items='center',
                                                      height='50px', max_height='75px', min_height='25px', 
                                                      width='200px', max_width='300px',  min_width='100px'))
HL_slider.observe(HL_changed, 'value')
                                
# Select Generic or Specific Isotopes
isotope = widgets.RadioButtons(options=['Generic', 'Rb/Sr'], value='Generic', description='Isotope:', 
                               disabled=False, 
                               layout=widgets.Layout(align_content='center', align_items='center',
                                                      height='75px', max_height='100px', min_height='50px', 
                                                      width='200px', max_width='300px',  min_width='100px'))
isotope.observe(isotope_changed, 'value')

# Describe slope
slope_label = widgets.Label(description='Slope', value = 'Slope: {0:.2f}'.format(slope))

# Describe current half-lives OR Age
HLlabel = widgets.Label (value='{0:.2f} Half-lives'.format(HL_slider.value))

controls = widgets.VBox( [isotope, HL_slider, HLlabel, slope_label], 
                        layout=widgets.Layout(align_content='center', align_items='center', 
                                              justify_content='flex-start', 
                                              height='500px', max_height='600px', min_height='400px', 
                                              width='250px', max_width='300px',  min_width='100px',
                                              overflow_x='hidden', overflow_y='hidden') )

display(widgets.HBox( [isochron, controls] ) )


